<a href="https://colab.research.google.com/github/Dhaneshkp/GraphML/blob/main/Fraudulent%20transactions-ne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

!pip install --upgrade certifi

!pip install --trusted-host files.pythonhosted.org optuna



In [3]:
os.listdir()

['.config',
 'fraudTest.csv',
 'fraudTrain.csv',
 'train_df_new.csv',
 'sample_data']

In [4]:
data=pd.read_csv("fraudTrain.csv")

In [5]:

def are_cc_numbers_unique(df, cc_column):

    try:
      # Check if the specified column exists in the DataFrame.
      if cc_column not in df.columns:
          print(f"Error: Column '{cc_column}' not found in the DataFrame.")
          return False

      return df[cc_column].nunique() == len(df)

    except Exception as e:
      print(f"An error occurred: {e}")
      return False


are_unique = are_cc_numbers_unique(data, 'cc_num')

if are_unique:
     print("All credit card numbers are unique.")
else:
     print("Some credit card numbers are duplicated.")

Some credit card numbers are duplicated.


In [6]:

import datetime

data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

# Extract the time of day as an integer representing minutes since midnight
data['time_of_day'] = (data['trans_date_trans_time'].dt.hour * 60) + data['trans_date_trans_time'].dt.minute


data['time_of_day_category'] = pd.cut(data['time_of_day'], bins=[0, 6*60, 12*60, 18*60, 24*60],
                                      labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)


data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,time_of_day,time_of_day_category
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,0,Night
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,0,Night
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,0,Night
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,1,Night
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,3,Night


In [7]:

data['unix_time_c'] = pd.to_datetime(data['unix_time'], unit='s')

# Extract the time of day as an integer representing minutes since midnight
data['time_of_day_uc'] = (data['unix_time_c'].dt.hour * 60) + data['unix_time_c'].dt.minute

data['time_of_day_category_uc'] = pd.cut(data['time_of_day_uc'], bins=[0, 6*60, 12*60, 18*60, 24*60],
                                      labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,time_of_day,time_of_day_category,unix_time_c,time_of_day_uc,time_of_day_category_uc
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,0,Night,2012-01-01 00:00:18,0,Night
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,0,Night,2012-01-01 00:00:44,0,Night
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,0,Night,2012-01-01 00:00:51,0,Night
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,1,Night,2012-01-01 00:01:16,1,Night
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,3,Night,2012-01-01 00:03:06,3,Night


In [8]:
data.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'time_of_day',
       'time_of_day_category', 'unix_time_c', 'time_of_day_uc',
       'time_of_day_category_uc'],
      dtype='object')

In [9]:
data.describe()

,Unnamed: 0,trans_date_trans_time,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,time_of_day,unix_time_c,time_of_day_uc
count,1.296675e+06,1296675,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1296675,1.296675e+06
mean,6.483370e+05,2019-10-03 12:47:28.070214144,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03,7.977868e+02,2012-10-03 05:53:56.726123008,7.977868e+02
min,0.000000e+00,2019-01-01 00:00:18,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00,0.000000e+00,2012-01-01 00:00:18,0.000000e+00
25%,3.241685e+05,2019-06-03 19:12:22.500000,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00,4.580000e+02,2012-06-03 19:12:22.500000,4.580000e+02
50%,6.483370e+05,2019-10-03 07:35:47,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00,8.480000e+02,2012-10-03 07:35:47,8.480000e+02
75%,9.725055e+05,2020-01-28 15:02:55.500000,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00,1.145000e+03,2013-01-28 15:02:55.500000,1.145000e+03
max,1.296674e+06,2020-06-21 12:13:37,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00,1.439000e+03,2013-06-21 12:13:37,1.439000e+03
std,3.743180e+05,NaN,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02,4.094252e+02,NaN,4.094252e+02


In [10]:
data.describe(include='object')

,merchant,category,first,last,gender,street,city,state,job,dob,trans_num
count,1296675,1296675,1296675,1296675,1296675,1296675,1296675,1296675,1296675,1296675,1296675
unique,693,14,352,481,2,983,894,51,494,968,1296675
top,fraud_Kilback LLC,gas_transport,Christopher,Smith,F,0069 Robin Brooks Apt. 695,Birmingham,TX,Film/video editor,1977-03-23,0b242abb623afc578575680df30655b9
freq,4403,131659,26669,28794,709863,3123,5617,94876,9779,5636,1


In [11]:
train_df = data.drop(['Unnamed: 0', 'trans_date_trans_time','trans_num', 'unix_time'],axis=1)


In [12]:
train_df=train_df.sample(frac=0.5,random_state=1)# if the data us indded by a person or a cc num then this sampling isnt appropriate

In [13]:
train_df['distance']= (train_df['lat'] - train_df['merch_lat'])**2 + (train_df['long'] - train_df['merch_long'])**2

In [14]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english') and not word.isdigit()])
    return text

# Apply cleaning to relevant columns
#train_df['merchant'] = train_df['merchant'].apply(clean_text)
#train_df['category'] = train_df['category'].apply(clean_text)
train_df['city_state'] = train_df['city'] + '_' + train_df['state']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
train_df['city_state'] = train_df['city'] + '_' + train_df['state']

In [16]:
train_df = train_df.dropna(subset=['dob'])

In [17]:
from datetime import datetime

# Function to calculate age
def calculate_age(dob):
    dob = str(dob)
    today = datetime.today()
    dob = datetime.strptime(dob, '%Y-%m-%d')
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age

# Apply the function to the 'dob' column
train_df['age'] = train_df['dob'].apply(calculate_age)


In [18]:
categorical_columns = ['category', 'gender', 'job','city','state','merchant','street','city_state']
for column in categorical_columns:
    train_df[column] = train_df[column].astype('category')

In [19]:
train_df.dtypes

,0
cc_num,int64
merchant,category
category,category
amt,float64
first,object
last,object
gender,category
street,category
city,category
state,category


In [20]:
train_df.drop(['merch_lat', 'merch_long','lat', 'long'],axis=1,inplace=True)

In [21]:
train_df.columns

Index(['cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender',
       'street', 'city', 'state', 'zip', 'city_pop', 'job', 'dob', 'is_fraud',
       'time_of_day', 'time_of_day_category', 'unix_time_c', 'time_of_day_uc',
       'time_of_day_category_uc', 'distance', 'city_state', 'age'],
      dtype='object')

In [22]:
train_df.drop(['street','zip','city_pop'],axis=1,inplace=True)

In [23]:
train_df.drop(['dob'],axis=1,inplace=True)

In [24]:
train_df.to_csv("train_df_new.csv")

In [25]:
train_df=pd.read_csv("train_df_new.csv")
train_df.drop('Unnamed: 0',axis=1,inplace=True)

In [26]:
!pip install pykeen

In [27]:
import networkx as nx

G = nx.Graph()

# Add nodes and edges based on the data
for index, row in train_df.iterrows():
    trans_id = f'trans_{index}'
    G.add_node(trans_id, label='transaction')
    G.add_node(row['merchant'], label='merchant')
    G.add_node(row['category'], label='category')
    G.add_node(row['city'], label='city')
    G.add_node(row['state'], label='state')
    G.add_edge(trans_id, row['merchant'], relationship='made_at')
    G.add_edge(trans_id, row['category'], relationship='belongs_to')
    G.add_edge(trans_id, row['city'], relationship='located_in')
    G.add_edge(trans_id, row['state'], relationship='in')
    G.add_node(row['cc_num'], label='cc_num')
    G.add_edge(trans_id, row['cc_num'], relationship='card')
    #G.add_node(row['job'], label='job')
    #G.add_edge(trans_id, row['job'], relationship='job')

In [ ]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from sklearn.model_selection import train_test_split
import torch
# Create triples for KG (subject, predicate, object)
triples = []
for index, row in train_df.iterrows():
    trans_id = f'trans_{index}'
    triples.append((trans_id, 'made_at', row['merchant']))
    triples.append((trans_id, 'belongs_to', row['category']))
    triples.append((trans_id, 'located_in', row['city']))
    triples.append((trans_id, 'in', row['state']))
    #triples.append((trans_id, 'job', row['job']))
    triples.append((trans_id, 'card', row['cc_num']))
triples = np.array(triples)
# Create a PyKEEN dataset
for i in range(25):
    print(triples[i])

# Check data types of the triples
print(type(triples[0][0]), type(triples[0][1]), type(triples[0][2]))

# Remove malformed entries
valid_triples = [triple for triple in triples if len(triple) == 3]
triples = np.array(valid_triples)

# Split the triples into training and testing sets
train_triples, test_triples = train_test_split(triples, test_size=0.2, random_state=42)
train_triples=triples
# Create TriplesFactory objects for training and testing sets
train_tf = TriplesFactory.from_labeled_triples(train_triples)
test_tf = TriplesFactory.from_labeled_triples(test_triples)

# Print TriplesFactory details for debugging
print(train_tf)
print(test_tf)

# Train a KG embedding model with both training and testing triples factories
result = pipeline(
    training=train_tf,
    testing=test_tf,
    model='TransE',
    training_kwargs=dict(num_epochs=10),
)
all_entities = train_tf.entity_to_id
embeddings = result.model.entity_representations[0](indices=torch.arange(train_tf.num_entities, device=result.model.device)).cpu().detach().numpy()

# Create a DataFrame with embeddings
embedding_df = pd.DataFrame(embeddings, index=all_entities.keys())

# Merge embeddings with labels
train_df['embedding'] = train_df.index.map(lambda x: embedding_df.loc[f'trans_{x}'].values)

['trans_0' 'made_at' 'fraud_Funk Group']
['trans_0' 'belongs_to' 'grocery_net']
['trans_0' 'located_in' 'Wilton']
['trans_0' 'in' 'ND']
['trans_0' 'card' '374497717543058']
['trans_1' 'made_at' 'fraud_Prosacco, Kreiger and Kovacek']
['trans_1' 'belongs_to' 'home']
['trans_1' 'located_in' 'Colton']
['trans_1' 'in' 'WA']
['trans_1' 'card' '4428154703770710']
['trans_2' 'made_at' 'fraud_Langworth, Boehm and Gulgowski']
['trans_2' 'belongs_to' 'shopping_net']
['trans_2' 'located_in' 'Brandon']
['trans_2' 'in' 'FL']
['trans_2' 'card' '213148039875802']
['trans_3' 'made_at' 'fraud_Conroy-Emard']
['trans_3' 'belongs_to' 'food_dining']
['trans_3' 'located_in' 'Alpharetta']
['trans_3' 'in' 'GA']
['trans_3' 'card' '2283743876903625']
['trans_4' 'made_at' 'fraud_Adams-Barrows']
['trans_4' 'belongs_to' 'health_fitness']
['trans_4' 'located_in' 'Bay City']
['trans_4' 'in' 'MI']
['trans_4' 'card' '4798937978667559470']
<class 'numpy.str_'> <class 'numpy.str_'> <class 'numpy.str_'>


INFO:pykeen.pipeline.api:Using device: None


TriplesFactory(num_entities=650973, num_relations=5, create_inverse_triples=False, num_triples=3241690)
TriplesFactory(num_entities=438333, num_relations=5, create_inverse_triples=False, num_triples=648338)


Training epochs on cpu:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/12663 [00:00<?, ?batch/s]

In [ ]:
!pip install python-louvain

from community import community_louvain

# Calculate the Louvain community structure
partition = community_louvain.best_partition(G)

# Add the community assignments to the dataframe
train_df['community'] = train_df.index.map(lambda x: partition.get(f'trans_{x}', -1))

# Print or further analyze the community assignments
print(train_df[['community']])

In [ ]:
train_df.describe()

In [ ]:
from sklearn.semi_supervised import LabelPropagation

# Ensure labels and graph_laplacian have the same dimension
labels = train_df['is_fraud'].values
# Get the indices of the nodes corresponding to transactions
transaction_nodes = [node for node in G.nodes() if str(node).startswith('trans_')]
# Subset the graph_laplacian to include only transaction nodes
graph_laplacian = nx.laplacian_matrix(G, nodelist=transaction_nodes).toarray()

label_prop_model = LabelPropagation()

# Fit the model with the adjusted graph_laplacian and labels
label_prop_model.fit(graph_laplacian, labels)

# Predict labels for transaction nodes
predicted_labels = label_prop_model.predict(graph_laplacian)

# Add the predicted labels to the dataframe
train_df['predicted_labels'] = predicted_labels

print(train_df[['community', 'predicted_labels']])

In [ ]:
train_df.info()

In [ ]:
train_df.dropna()

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == 'object' and col != 'embedding':  # Skip 'embedding' column
        train_df[col] = train_df[col].astype('category')

In [ ]:
train_df=train_df.dropna()

In [ ]:
train_df

In [ ]:
train_df.to_csv("train_df_new.csv")

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Fit and transform the 'gender' column in both training and validation data
train_df['gender_encoded'] = le.fit_transform(train_df['gender'])


X =  train_df[['amt', 'distance', 'age', 'gender_encoded','community','predicted_labels','job']]
y = train_df['is_fraud']



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model = xgb.XGBClassifier(random_state=42, enable_categorical=True)
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)


print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, precision_score, recall_score, f1_score

y_val_pred_proba = model.predict_proba(X_val)[:, 1]
cutoffs = np.linspace(0, 1, 101)
precision = []
recall = []
f1 = []

for cutoff in cutoffs:
    y_val_pred = (y_val_pred_proba >= cutoff).astype(int)
    precision.append(precision_score(y_val, y_val_pred))
    recall.append(recall_score(y_val, y_val_pred))
    f1.append(f1_score(y_val, y_val_pred))

plt.figure(figsize=(10, 6))
plt.plot(cutoffs, precision, label='Precision')
plt.plot(cutoffs, recall, label='Recall')
plt.plot(cutoffs, f1, label='F1 Score')
plt.xlabel('Cutoff Probability')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score vs. Cutoff Probability')
plt.legend()
plt.grid(True)
plt.show()


best_cutoff = cutoffs[np.argmax(f1)]
print(f"Best cutoff for F1 Score: {best_cutoff}")

y_val_pred = (y_val_pred_proba >= best_cutoff).astype(int)
print(classification_report(y_val,y_val_pred))


xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

In [ ]:

df_majority = train_df[train_df.is_fraud==0]
df_minority = train_df[train_df.is_fraud==1]

# Upsample minority class
from sklearn.utils import resample
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=int(0.05*len(train_df)),    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
train_df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
train_df_upsampled.is_fraud.value_counts()



In [ ]:

le = LabelEncoder()
train_df_upsampled['gender_encoded'] = le.fit_transform(train_df_upsampled['gender'])

X =  train_df_upsampled[['amt', 'distance', 'age', 'gender_encoded','community']]
y = train_df_upsampled['is_fraud']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model = xgb.XGBClassifier(random_state=42, enable_categorical=True)
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)

print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

y_val_pred_proba = model.predict_proba(X_val)[:, 1]
cutoffs = np.linspace(0, 1, 101)
precision = []
recall = []
f1 = []

for cutoff in cutoffs:
    y_val_pred = (y_val_pred_proba >= cutoff).astype(int)
    precision.append(precision_score(y_val, y_val_pred))
    recall.append(recall_score(y_val, y_val_pred))
    f1.append(f1_score(y_val, y_val_pred))

plt.figure(figsize=(10, 6))
plt.plot(cutoffs, precision, label='Precision')
plt.plot(cutoffs, recall, label='Recall')
plt.plot(cutoffs, f1, label='F1 Score')
plt.xlabel('Cutoff Probability')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score vs. Cutoff Probability')
plt.legend()
plt.grid(True)
plt.show()

best_cutoff = cutoffs[np.argmax(f1)]
print(f"Best cutoff for F1 Score: {best_cutoff}")

y_val_pred = (y_val_pred_proba >= best_cutoff).astype(int)
print(classification_report(y_val,y_val_pred))

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

In [ ]:
scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1])

model = xgb.XGBClassifier(early_stopping_rounds =5 ,eval_metric='auc', random_state=42,enable_categorical=True,scale_pos_weight=scale_pos_weight)
model.fit(X_train, y_train,eval_set=[(X_val, y_val)])

y_val_pred = model.predict(X_val)


print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()


In [ ]:
test_df = pd.read_csv('fraudTest.csv')

In [ ]:
test_df['trans_date_trans_time'] = pd.to_datetime(test_df['trans_date_trans_time'])
test_df['time_of_day'] = (test_df['trans_date_trans_time'].dt.hour * 60) + test_df['trans_date_trans_time'].dt.minute
test_df['time_of_day_category'] = pd.cut(test_df['time_of_day'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
test_df['unix_time_c'] = pd.to_datetime(test_df['unix_time'], unit='s')
test_df['time_of_day_uc'] = (test_df['unix_time_c'].dt.hour * 60) + test_df['unix_time_c'].dt.minute
test_df['time_of_day_category_uc'] = pd.cut(test_df['time_of_day_uc'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

test_df = test_df.drop(['Unnamed: 0', 'trans_date_trans_time','trans_num', 'unix_time'],axis=1)
test_df['distance']= (test_df['lat'] - test_df['merch_lat'])**2 + (test_df['long'] - test_df['merch_long'])**2
test_df['merchant'] = test_df['merchant'].apply(clean_text)
test_df['category'] = test_df['category'].apply(clean_text)
test_df['city_state'] = test_df['city'] + '_' + test_df['state']
test_df = test_df.dropna(subset=['dob'])
test_df['age'] = test_df['dob'].apply(calculate_age)

categorical_cols = ['category', 'gender', 'job','city','state','merchant','street','city_state']
for col in categorical_cols:
    test_df[col] = test_df[col].astype('category')

test_df.drop(['merch_lat', 'merch_long','lat', 'long','street','zip','city_pop','dob'],axis=1,inplace=True)


# Create graph for test data
G_test = nx.Graph()
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    G_test.add_node(trans_id, label='transaction')
    G_test.add_node(row['merchant'], label='merchant')
    G_test.add_node(row['category'], label='category')
    G_test.add_node(row['city'], label='city')
    G_test.add_node(row['state'], label='state')
    G_test.add_edge(trans_id, row['merchant'], relationship='made_at')
    G_test.add_edge(trans_id, row['category'], relationship='belongs_to')
    G_test.add_edge(trans_id, row['city'], relationship='located_in')
    G_test.add_edge(trans_id, row['state'], relationship='in')
    G_test.add_node(row['cc_num'], label='cc_num')
    G_test.add_edge(trans_id, row['cc_num'], relationship='card')

# Community detection for test data
partition_test = community_louvain.best_partition(G_test)
test_df['community'] = test_df.index.map(lambda x: partition_test.get(f'trans_{x}', -1))

# Label Propagation for test data (using the trained model)
transaction_nodes_test = [node for node in G_test.nodes() if str(node).startswith('trans_')]
graph_laplacian_test = nx.laplacian_matrix(G_test, nodelist=transaction_nodes_test).toarray()
predicted_labels_test = label_prop_model.predict(graph_laplacian_test) # Use the trained model here
test_df['predicted_labels'] = predicted_labels_test


# Feature Engineering and Prediction
test_df['gender_encoded'] = le.transform(test_df['gender']) # Use the same LabelEncoder
X_test = test_df[['amt', 'distance', 'age', 'gender_encoded', 'community', 'predicted_labels', 'job']]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_pred_proba >= best_cutoff).astype(int) # Use the best cutoff from training

# Evaluate the model on the test data
print(classification_report(test_df['is_fraud'], y_test_pred)) # Assuming 'is_fraud' column exists in test_df
print(confusion_matrix(test_df['is_fraud'], y_test_pred))

In [ ]:
# prompt: evaluate cgb model on  the test_df incliding the graph community and lablr

# Preprocess test_df similar to train_df
test_df['trans_date_trans_time'] = pd.to_datetime(test_df['trans_date_trans_time'])
test_df['time_of_day'] = (test_df['trans_date_trans_time'].dt.hour * 60) + test_df['trans_date_trans_time'].dt.minute
test_df['time_of_day_category'] = pd.cut(test_df['time_of_day'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
test_df['unix_time_c'] = pd.to_datetime(test_df['unix_time'], unit='s')
test_df['time_of_day_uc'] = (test_df['unix_time_c'].dt.hour * 60) + test_df['unix_time_c'].dt.minute
test_df['time_of_day_category_uc'] = pd.cut(test_df['time_of_day_uc'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
test_df['distance'] = (test_df['lat'] - test_df['merch_lat'])**2 + (test_df['long'] - test_df['merch_long'])**2
test_df['merchant'] = test_df['merchant'].apply(clean_text)
test_df['category'] = test_df['category'].apply(clean_text)
test_df['city_state'] = test_df['city'] + '_' + test_df['state']
test_df['age'] = test_df['dob'].apply(calculate_age)

# Select relevant columns for prediction
test_X = test_df[['amt', 'distance', 'age', 'gender', 'city','state','job']] # Include necessary features

# Encode categorical features in test data using the same label encoders as training
for col in ['gender', 'job','city', 'state']:
  le = LabelEncoder()
  le.fit(train_df[col]) # fit on train data
  test_df[col+'_encoded'] = le.transform(test_df[col])

# Use the same features used in your trained model
test_X = test_df[['amt', 'distance', 'age', 'gender_encoded','community']] # Example features, adjust as needed
test_df['community'] = test_df.index.map(lambda x: partition.get(f'trans_{x}', -1))
# Make predictions
test_y_pred = model.predict(test_X)
test_y_pred



##### Use stacking

In [ ]:
!pip install scikit-learn==1.2.2
!pip install xgboost==1.7.5
!pip install lightgbm==3.3.5
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression



base_learners = [
    ('lgbm',lgbm.LGBMClassifier(random_state=42)),
    ('xgb', xgb.XGBClassifier(random_state=42, enable_categorical=True))
]

meta_learner = LogisticRegression()
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner)

stacking_model.fit(X_train, y_train)


In [ ]:
base_learners = [
    #('lgbm',lgbm.LGBMClassifier(random_state=42)),
    ('xgb', xgb.XGBClassifier(random_state=42, enable_categorical=True))
]

meta_learner = LogisticRegression()
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner)

stacking_model.fit(X_train, y_train)

In [ ]:
#model = xgb.XGBClassifier(early_stopping_rounds=10, eval_metric='auc',enable_categorical=True)
#model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

# Make predictions on the validation set
y_val_pred = stacking_model.predict(X_val)

print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")
